In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd    #89, 91, 92
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('nuovo_dataset_totale.csv')

# Filter the dataset based on the 'genre' column
filtered_df = dataset
filtered_df = filtered_df.drop(['id'], axis=1)

# Function to transform strings into lists of floats
def transform_to_list(lst_str):
    lst_str = lst_str.strip('][').split(', ')
    return [float(x) for x in lst_str]

# Apply the function to the 'popularity_artist' column
filtered_df['popularity_artist'] = filtered_df['popularity_artist'].apply(transform_to_list)

# Calculate the mean of each list
filtered_df['popularity_artist'] = round(filtered_df['popularity_artist'].apply(np.mean), 2)

# Define the target variables
y = filtered_df[['popularity']]

# Drop unnecessary columns from the feature matrix
X = filtered_df.drop(['name', 'Unnamed: 0', 'artists', 'disc_number', 'album_type', 'album_release_date',
                      'album_release_date_precision', 'album_name', 'genre',
                      'popularity'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)

# Normalize the feature matrix
norm = StandardScaler()
norm.fit(X_train)
X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)


# MLRegressor

In [3]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV

# Create an MLP Regressor
mlp_regressor = MLPRegressor()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],  # Various hidden layer sizes
    'activation': ['logistic', 'tanh', 'relu'],  # Activation functions
    'solver': ['adam', 'sgd', 'lbfgs'],  # Solvers for optimization
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [200, 300, 1000], 
    'early_stopping': [True],
    'n_iter_no_change': [20,30,40], # L2 regularization parameter
    'momentum': [0.1, 0.001, 0.0001],
    'learning_rate': ['constant', 'adaptive'],  # Learning rate schedule
}

# Create a GridSearchCV object
grid_search = RandomizedSearchCV(mlp_regressor, param_grid, cv=5, n_iter= 30, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_norm, y_train)

: 

In [ ]:
# Get the best estimator and its parameters
best_mlp_regressor = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
print("Best Score:", grid_search.best_score_)


# Evaluate the best model on the test set
y_pred = best_mlp_regressor.predict(X_test_norm)
# Calculate the evaluation metrics for each target variable
# You can use metrics like mean_squared_error, r2_score, etc.
